In [1]:
import sys
sys.path.append('/mnt/workspace')

In [2]:
from py_file.M_attri import Att
from py_file.Get_Data import DATA
from py_file.data_set import MyDataSet
from torch.utils.data import DataLoader,Dataset,random_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import numpy as np
import matplotlib.pyplot as plt
import os
import math

In [3]:
# 定义训练的设备
if torch.cuda.is_available():
    device = torch.device('cuda:0') # 只有一张显卡的话，'cuda'和'cuda:0'是一样的
else:
    device = torch.device('cpu')

print(f'使用的设备为：{device}')

使用的设备为：cuda:0


In [4]:
# 这里加载属性是为了方便获取类别的种类
attri = Att()
attri.compute_mul_defect_att()

In [5]:
trans = transforms.Resize(224)  # ResNet模型适合的图片大小为224x244
# 输入的张量需要带着批次维度和通道维度

# 准备模型以及数据

In [6]:
model = torch.load(f='pseudo_supervised.pth')
model.eval()
model = model.to(device)

In [7]:
test_data_path = '/mnt/workspace/DATA/test_WM.npz'
test_data = np.load(test_data_path)

test_label = test_data['label_name']
test_wm = test_data['denoise_wm']
test_wm_tensor = trans(torch.reshape(torch.tensor(test_wm, dtype=torch.float32),(len(test_wm),1,52,52)))

print(test_wm_tensor.shape, test_label.shape)

torch.Size([7405, 1, 224, 224]) (7405,)


## 将标签的多热编码改为独热编码

In [8]:
label_order = list(attri.total_defect_att.keys())
print(label_order)
print(len(label_order))

['Center', 'Donut', 'Edge_loc', 'Loc', 'Edge_ring', 'Scratch', 'Random', 'Nearfull', 'Normal', 'C+EL', 'C+ER', 'C+L', 'C+S', 'D+EL', 'D+ER', 'D+L', 'D+S', 'EL+L', 'EL+S', 'ER+L', 'ER+S', 'L+S', 'C+EL+L', 'C+EL+S', 'C+ER+L', 'C+ER+S', 'C+L+S', 'D+EL+L', 'D+EL+S', 'D+ER+L', 'D+ER+S', 'D+L+S', 'EL+L+S', 'ER+L+S', 'C+EL+L+S', 'C+ER+L+S', 'D+EL+L+S', 'D+ER+L+S']
38


In [9]:
label_map_dict = {}  # 字符标签映射为one-hot的字典
for i in range(len(label_order)):
    oh = [0 for _ in range(len(label_order))]
    oh[i] = 1
    label_map_dict[label_order[i]] = oh
print(label_map_dict)

{'Center': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'Donut': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'Edge_loc': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'Loc': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'Edge_ring': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'Scratch': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'Random': [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'Nearfull': [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [10]:
test_label_oh = []
for l in test_label:
    test_label_oh.append(label_map_dict[l])

test_label = torch.tensor(np.array(test_label_oh), dtype=torch.float32)
print(test_wm_tensor.shape, test_label.shape)

torch.Size([7405, 1, 224, 224]) torch.Size([7405, 38])


## 将测试集划分

In [11]:
test_label_oh = test_data['label_one_hot']
# 通过one_hot标签给数据分为单缺陷、双缺陷、三缺陷以及四缺陷

test_single_wm = []  # 先定义列表，然后转换为tensor
test_single_label = []

test_two_wm = []
test_two_label = []

test_three_wm = []
test_three_label = []

test_four_wm = []
test_four_label = []

for i in range(len(test_label_oh)):
    if test_label_oh[i].sum() <= 1:
        test_single_wm.append(np.array(test_wm_tensor[i]))
        test_single_label.append(test_label[i])

    elif test_label_oh[i].sum() == 2:
        test_two_wm.append(np.array(test_wm_tensor[i]))
        test_two_label.append(test_label[i])

    elif test_label_oh[i].sum() == 3:
        test_three_wm.append(np.array(test_wm_tensor[i]))
        test_three_label.append(test_label[i])

    elif test_label_oh[i].sum() == 4:
        test_four_wm.append(np.array(test_wm_tensor[i]))
        test_four_label.append(test_label[i])


del test_data

In [12]:
test_size = len(test_wm_tensor)

# 定义测试需要的函数

In [13]:
# 预测outputs中预测正确的个数
def predict_num(outputs,labels_oh):
    right = 0
    for i in range(len(outputs)):
        if np.argmax(outputs[i]) == np.argmax(labels_oh[i]):
            right += 1
    
    return right

def get_acc(model, dataloader,data_size):
    model.eval()
    total_right_num = 0  # 记录正确的总个数
    with torch.no_grad():   # 这里要进行验证，不需要修改参数，所以不计算梯度
        for batch_idx, (inputs, labels_oh) in enumerate(dataloader):
            inputs = inputs.to(device)
            labels_oh = labels_oh.to(device)

            outputs = model(inputs)
            outputs = torch.softmax(outputs, dim=1)

            outputs = outputs.cpu().numpy()
            labels_oh = labels_oh.cpu().numpy()
            
            right_num = predict_num(outputs,labels_oh)
            total_right_num += right_num
    acc = total_right_num / data_size
    return acc

In [14]:
# 展示混淆矩阵的函数
from sklearn.metrics import confusion_matrix
import seaborn as sns
import pandas as pd

def show_cmat(true_label, predict_label, label_order,typ = 'per', size = 10):
    # 生成混淆矩阵
    cm = confusion_matrix(true_label, predict_label, labels=label_order)  # 按照label_order中的顺序给数据排序
    # 将混淆矩阵中的数字转换为百分比
    # 通过将cm中的每个元素除以其所在行的总和（即该类的真实样本数），再乘以100，将其转换为百分比。
    cm_percentage = cm / cm.sum(axis=1, keepdims=True) * 100

    plt.figure(figsize=(size,size))  # 这个要定义在sns.heatmap()前面才可以定义大小
    # 使用seaborn来可视化混淆矩阵
    if typ == 'per':
        # fmt=".1f"参数设置注释的格式为保留一位小数的浮点数, cmap="Blues"指定了颜色映射
        sns.heatmap(pd.DataFrame(cm_percentage, index=label_order, columns=label_order), annot=True, fmt=".1f", cmap="Blues")
        plt.title('Confusion Matrix in Percentage')
    else:
        # fmt="d"表示格式化为整数，
        sns.heatmap(pd.DataFrame(cm, index=label_order, columns=label_order), annot=True, fmt="d", cmap="Blues")
        plt.title('Confusion Matrix in Number')
    
    plt.xlabel('Predicted')
    plt.ylabel('True')
    # 显示图形
    plt.show()

# 获取真实标签
def predict_labels(outputs, labels_oh, label_order):
    pred_labels = []
    rel_labels = []
    for i in range(len(outputs)):
        pred_labels.append(label_order[np.argmax(outputs[i])])
        rel_labels.append(label_order[np.argmax(labels_oh[i])])
    return rel_labels, pred_labels

def get_labels(model, dataloader, label_order):
    label_true = []
    label_pre = []
    with torch.no_grad(): 
        for imgs,labels in dataloader:  
            imgs = imgs.to(device)
            labels = labels.to(device)
            
            outputs = model(imgs)

            outputs = outputs.cpu().numpy()
            labels = labels.cpu().numpy()

            y_true,y_pre = predict_labels(outputs, labels, label_order)
            
            label_true = label_true + y_true
            label_pre = label_pre + y_pre

    return label_true,label_pre

# 开始测试

## 所有故障

In [15]:
total_dataset = MyDataSet(test_wm_tensor,test_label)
test_size = len(total_dataset)
total_loader = DataLoader(total_dataset, batch_size=32, shuffle=False)
del test_wm_tensor

In [16]:
acc = get_acc(model,total_loader,test_size)
print(acc)

0.6550979068197164


## 混合故障

In [17]:
test_mul_wm = test_two_wm + test_three_wm + test_four_wm
test_mul_label = test_two_label + test_three_label + test_four_label

In [18]:
mul_wm_tensor = torch.tensor(np.array(test_mul_wm), dtype=torch.float32)
test_mul_label = torch.tensor(np.array(test_mul_label), dtype=torch.float32)
mul_dataset = MyDataSet(mul_wm_tensor,test_mul_label)
mul_size = len(mul_dataset)
mul_loader = DataLoader(mul_dataset, batch_size=32, shuffle=False)
del mul_wm_tensor

In [19]:
acc = get_acc(model,mul_loader,mul_size)
print(acc)

0.5627586206896552


## 单故障

In [20]:
single_wm_tensor = torch.tensor(np.array(test_single_wm), dtype=torch.float32)
test_single_label = torch.tensor(np.array(test_single_label), dtype=torch.float32)
single_dataset = MyDataSet(single_wm_tensor,test_single_label)
single_size = len(single_dataset)
single_loader = DataLoader(single_dataset, batch_size=32, shuffle=False)
del single_wm_tensor

In [21]:
acc = get_acc(model,single_loader,single_size)
print(acc)

0.9887850467289719


## 二故障

In [22]:
two_wm_tensor = torch.tensor(np.array(test_two_wm), dtype=torch.float32)
test_two_label = torch.tensor(np.array(test_two_label), dtype=torch.float32)
two_dataset = MyDataSet(two_wm_tensor,test_two_label)
two_size = len(two_dataset)
two_loader = DataLoader(two_dataset, batch_size=32, shuffle=False)
del two_wm_tensor

In [23]:
acc = get_acc(model,two_loader,two_size)
print(acc)

0.6484615384615384


## 三故障

In [24]:
three_wm_tensor = torch.tensor(np.array(test_three_wm), dtype=torch.float32)
test_three_label = torch.tensor(np.array(test_three_label), dtype=torch.float32)
three_dataset = MyDataSet(three_wm_tensor,test_three_label)
three_size = len(three_dataset)
three_loader = DataLoader(three_dataset, batch_size=32, shuffle=False)
del three_wm_tensor

In [25]:
acc = get_acc(model,three_loader,three_size)
print(acc)

0.5083333333333333


## 四故障

In [26]:
four_wm_tensor = torch.tensor(np.array(test_four_wm), dtype=torch.float32)
test_four_label = torch.tensor(np.array(test_four_label), dtype=torch.float32)
four_dataset = MyDataSet(four_wm_tensor,test_four_label)
four_size = len(four_dataset)
four_loader = DataLoader(four_dataset, batch_size=32, shuffle=False)
del four_wm_tensor

In [27]:
acc = get_acc(model,four_loader,four_size)
print(acc)

0.4475
